## Fabric Gateway Client
This notebook interacts with our Hyperledger Fabric blockchain, using our JavaScript module, as to simulate a Jupyter extension (i.e. the frontend).
Note that the code is executed on the backend (using a Jupyter kernel), whereas a Jupyter extension would run this code on the frontend (i.e. the web browser). This has two major consequences:
- the traffic between this client and Fabric will occur via the backend (i.e. IP routing, DNS, TLS, etc. will take place within that context);
- and the user's private key is generated and stored on the backend (rather than never leaving the web browser's memory). As long as JupyterLab is executed on the *same* system (e.g. your laptop) this does not matter (both the backend and frontend are running on the same host), but be aware when tunneling JupyterLab.

### Prerequisites
We use 'IJavascript' to run JavaScript from JupyterLab (as a kernel / on the JupyterLab backend. We also install the node version specified by Fabric's documentation (v14).

In [2]:
!mamba install --yes make cxx-compiler nodejs=14.18.3


                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (0.22.0) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['make', 'cxx-compiler', 'nodejs=14.18.3']

conda-forge/linux-64                                   

In [3]:
!npm install -g ijavascript
!ijsinstall

npm WARN deprecated uuid@3.4.0: Please upgrade  to version 7 or higher.  Older versions may use Math.random() in certain circumstances, which is known to be problematic.  See https://v8.dev/blog/math-random for details.
/opt/conda/bin/ijsconsole -> /opt/conda/lib/node_modules/ijavascript/bin/ijsconsole.js:refresh-package-jsmK
/opt/conda/bin/ijs -> /opt/conda/lib/node_modules/ijavascript/bin/ijavascript.js
/opt/conda/bin/ijsinstall -> /opt/conda/lib/node_modules/ijavascript/bin/ijsinstall.js
/opt/conda/bin/ijsnotebook -> /opt/conda/lib/node_modules/ijavascript/bin/ijsnotebook.js
/opt/conda/bin/ijskernel -> /opt/conda/lib/node_modules/ijavascript/lib/kernel.js

> zeromq@5.2.8 install /opt/conda/lib/node_modules/ijavascript/node_modules/zeromq
> node-gyp-build || npm run build:libzmq

+ ijavascript@5.2.1
added 8 packages from 31 contributors in 6.861s


#### Admin tasks
The organization's (Fabric and IPFS) administrator has to register a user and provision the IPFS network configuration.

**User registration**  
The organization's Fabric CA admin registers user by using the command below on the Docker host (which will use our Fabric CA client application):

**Network provisioning**  
The organization's Fabric admin uploads the IPFS private network configuration to the Fabric blockchain by using the command below on the Docker host (by using our IPFS smart contract):

### Node.js module setup

In [4]:
!npm install --only=prod ./*.tgz

npm WARN deprecated querystring@0.2.0: The querystring API is considered Legacy. new code should use the URLSearchParams API instead.
        ......] / install:@protobufjs/pool: info lifecycle @protobufjs/poo
> pkcs11js@1.3.0 install /home/jovyan/work/local/node_modules/pkcs11js
> node-gyp rebuild

make: Entering directory '/home/jovyan/work/local/node_modules/pkcs11js/build'
  CXX(target) Release/obj.target/pkcs11/src/main.o
In file included from ../../nan/nan.h:60,
                 from ../src/main.cpp:1:
/home/jovyan/.cache/node-gyp/14.18.3/include/node/node.h:787:7: warning: cast between incompatible function types from 'void (*)(Nan::ADDON_REGISTER_FUNCTION_ARGS_TYPE)' {aka 'void (*)(v8::Local<v8::Object>)'} to 'node::addon_register_func' {aka 'void (*)(v8::Local<v8::Object>, v8::Local<v8::Value>, void*)'} []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wcast-function-type-Wcast-function-type]8;;]
  787 |       (node::addon_register_func) (regfunc),         

#### Switch to JavaScript/Node.js Jupyter kernel and import the module

In [1]:
const fabric = require('jc-fabricgw-client');

### Configuration
We use a configuration file to configure our connection to the Fabric blockchain (using the Fabric Gateway service that was added in Fabric version 2.4). 

In [2]:
let config;
fabric.getConfig('./fabric-client-config.yaml').then((result) => {config = result;}); // promises/async in IJavascript: https://github.com/n-riesco/ijavascript/issues/268

Configuration File: ./fabric-client-config.yaml


Promise { <pending> }

In [3]:
console.log(config);

{
  organization: 'orgA',
  mspId: 'MSPorgA',
  identity: 'orgAuser',
  idCertFile: './id/orgAuser.crt',
  idKeyFile: './id/orgAuser.key',
  caEndpoint: 'https://ca.orga.fabric.localhost:7054',
  caTlsCertFile: '../crypto-config/fabric/peerOrganizations/orga.fabric.localhost/ca/ca.orga.fabric.localhost-cert.pem',
  caName: 'ca.orga.fabric.localhost',
  gatewayEndpoint: 'peer0.orga.fabric.localhost:7051',
  gatewayTlsCertFile: '../crypto-config/fabric/peerOrganizations/orga.fabric.localhost/peers/peer0.orga.fabric.localhost/tls/ca.crt',
  gatewayHostAlias: 'peer0.orga.fabric.localhost',
  channel: 'consortium-chain',
  chaincode: 'consortium-cc-ipfs'
}


#### User enrollment
We have to enroll our previously registered user identity (i.e. generate our public/private key pair). Note that our credentials are stored on the storage of the JupyterLab backend.

In [4]:
const CAregisterSecret = 'syftizGMliAX';
fabric.execEnroll(config, CAregisterSecret);

 Enrolling... 


Promise { <pending> }

Enrollment complete!


#### Set up the connection to the Fabric gateway peer
Now that we have set up the configuration and our identity, we can connect to Fabric. Note that our configuration specifies the Fabric Gateway endpoint we want to connect to, the channel/chain/network, and the smart contract we want to use.

In [4]:
let connectionDetails;
fabric.createConnection(config).then((result) => {connectionDetails = result;});

Promise { <pending> }

In [5]:
console.log(connectionDetails);

{
  gRpcClient: Client {
    [Symbol()]: [],
    [Symbol()]: [],
    [Symbol()]: undefined,
    [Symbol()]: ChannelImplementation {
      credentials: [SecureChannelCredentialsImpl],
      options: [Object],
      connectivityState: 0,
      currentPicker: [QueuePicker],
      configSelectionQueue: [],
      pickQueue: [],
      connectivityStateWatchers: [],
      configSelector: null,
      currentResolutionError: null,
      channelzEnabled: true,
      callTracker: [ChannelzCallTracker],
      childrenTracker: [ChannelzChildrenTracker],
      originalTarget: 'peer0.orga.fabric.localhost:7051',
      callRefTimer: Timeout {
        _idleTimeout: 2147483647,
        _idlePrev: [TimersList],
        _idleNext: [TimersList],
        _idleStart: 40360,
        _onTimeout: [Function (anonymous)],
        _timerArgs: undefined,
        _repeat: 2147483647,
        _destroyed: false,
        [Symbol(refed)]: false,
        [Symbol(kHasPrimitive)]: false,
        [Symbol(asyncId)]: 35,
    

#### Configure IPFS
Get the configuration for our local IPFS node from our Fabric blockchain (i.e. the IPFS networks we have access to and their related IPFS bootstrap nodes and network key). We will use the IPFS client Python module in the [IPFS notebook](./IPFS.ipynb#Join-the-IPFS-network) to interact with IPFS.

In [6]:
let networks;
fabric.listAllNetworks(connectionDetails.contract, 'MSPorgA').then((result) => {networks = result;});

Promise { <pending> }

In [7]:
console.log(networks);
for (const net of networks) {
  console.log(JSON.parse(net));
}

[{"ACL":{"MSPs":{"MSPorgA":"r"},"Users":{}},"BootstrapNodes":"/dns4/peer0.pnet0.orga.ipfs.localhost/tcp/4001/p2p/12D3KooWEq2SRZg8zYMjm3vp8G1xettJpZvjF62n58iJu7jhKdzm;/dns4/peer0.pnet0.orgb.ipfs.localhost/tcp/4001/p2p/12D3KooWLy5UaZMMR8rtvxHFa3L9LhvjWwdMBtP9e7BpHjvooiPD;/dns4/peer0.pnet0.orgc.ipfs.localhost/tcp/4001/p2p/12D3KooWByqyhou8Mj5YXTAojVE9TMLqkh6cKqDaEGX4HN5ML3PW","ClusterPinningService":{"https://cluster0.pnet0.orga.ipfs.localhost:9097":{"password":"325298731aB2022aFF0964813762fC","user":"orga"}},"ID":"pnet0","NetKey":"/key/swarm/psk/1.0.0/\n/base16/\n63e8d44cb8d738ece5681d42dc918ff882cbe28458d81f64e764f95e3f77929f","Owner":{"ID":"orgadmin","MSPId":"MSPorgA"}}]


### Add a data description
After having [added](./IPFS.ipynb#Add-a-file) a file via IPFS, we will upload its metadata to Fabric via our smart contract. Note that we give access to all users of Organization C (Fabric identities belonging to MSP 'MSPorgC').

In [1]:
let filename = 'testorga.txt';
let network = 'pnet0';
let cid = 'QmQGtBqmztM5KtUCkWWXYooobJhxWfKRZrvUtmXgQxx4Ch';
let cipher = 'ChaCha20';
let cryptKey = 'EDBNNOs1dDeKXk4Id+XCqg+di/26cm3DR1I5X+Nl//o=';
let chunkSize = '10485760';
let acl = '{"Users":{},"MSPs":{"MSPorgC":"r"}}';
let addDataDescr;
fabric.createData(connectionDetails.contract, filename, network, cid, cipher, cryptKey, chunkSize, acl).then((result) => {addDataDescr = result;});

ReferenceError: fabric is not defined

In [7]:
console.log(addDataDescr);
console.log(JSON.parse(addDataDescr));

undefined


### Retrieve a data description
We can retrieve a file's metadata from our Fabric channel/blockchain, provided we know its name and we have been granted access by the file's owner via the smart contract. Once we have obtained the metadata, we can then pass it to IPFS to [download](./IPFS.ipynb#Retrieve-a-file) the file itself.

In [8]:
let key = 'orgAuser@MSPorgA/testorga.txt';
let retrieveDataDescr;
fabric.readData(connectionDetails.contract, key).then((result) => {retrieveDataDescr = result;});

Promise { <pending> }

In [14]:
console.log(retrieveDataDescr);
console.log(JSON.parse(retrieveDataDescr));

{"ACL":{"MSPs":{"MSPorgC":"r"},"Users":{}},"CID":"QmQGtBqmztM5KtUCkWWXYooobJhxWfKRZrvUtmXgQxx4Ch","ChunkSize":"10485760","CryptCipher":"ChaCha20","CryptKey":"EDBNNOs1dDeKXk4Id+XCqg+di/26cm3DR1I5X+Nl//o=","ID":"testorga.txt","NetworkId":"pnet0","Owner":{"ID":"orgAuser","MSPId":"MSPorgA"}}
{
  ACL: { MSPs: { MSPorgC: 'r' }, Users: {} },
  CID: 'QmQGtBqmztM5KtUCkWWXYooobJhxWfKRZrvUtmXgQxx4Ch',
  ChunkSize: '10485760',
  CryptCipher: 'ChaCha20',
  CryptKey: 'EDBNNOs1dDeKXk4Id+XCqg+di/26cm3DR1I5X+Nl//o=',
  ID: 'testorga.txt',
  NetworkId: 'pnet0',
  Owner: { ID: 'orgAuser', MSPId: 'MSPorgA' }
}


### List data descriptions
We can also list all the data descriptions, that we have access to, for the specified user:

In [ ]:
let key = 'orgAuser@MSPorgA';
let listDataDescr;
fabric.listAllData(connectionDetails.contract, key).then((result) => {listDataDescr = result;});

In [ ]:
console.log(listDataDescr);
for (const dataDescr of listDataDescr) {
  console.log(JSON.parse(dataDescr));
}

### Delete a data description
We can delete a file's metadata from our Fabric channel/blockchain, provided we know its name and we have permission to do so (because we are the data description's owner or have write access).

In [ ]:
let key = 'orgAuser@MSPorgA/testorga.txt';
let delDataDescr;
fabric.deleteData(connectionDetails.contract, key).then((result) => {delDataDescr = result;});

In [ ]:
console.log(delDataDescr);

#### Close the Fabric connection

In [9]:
fabric.closeConnection(connectionDetails.gateway, connectionDetails.gRpcClient);

Promise { undefined }